In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras import layers
 
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
SEED_VALUE = 42
 
# Fix seed to make training deterministic.
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

0. CRIM - per capita crime rate by town
1. ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
2. INDUS - proportion of non-retail business acres per town.
3. CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
4. NOX - nitric oxides concentration (parts per 10 million)
5. RM - average number of rooms per dwelling
6. AGE - proportion of owner-occupied units built prior to 1940
7. DIS - weighted distances to five Boston employment centres
8. RAD - index of accessibility to radial highways
9. TAX - full-value property-tax rate per $10,000

In [ ]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()
 
print(X_train.shape)
print("\n")
print("Input features: ", X_train[0])
print("\n")
print("Output target: ", y_train[0])

In [ ]:
boston_features = {
    'Average Number of Rooms':5,
}
 
X_train_1d = X_train[:, boston_features['Average Number of Rooms']]
X_test_1d = X_test[:, boston_features['Average Number of Rooms']]
X_train_2d = X_train[:, 5:7]
X_test_2d = X_test[:, 5:7]

In [ ]:
print(X_train.shape)
print(X_train_2d.shape)

In [ ]:
plt.figure(figsize=(15, 5))
 
plt.xlabel('Average Number of Rooms')
plt.ylabel('Median Price [$K]')
plt.grid("on")
plt.scatter(X_train[:,5:6].flatten(), y_train, color='green',alpha=0.5);

In [ ]:
model = Sequential()
 
# Define the model consisting of a single neuron.
#model.add(Dense(units=1, input_shape=(1,)))
model.add(tf.keras.layers.Dense(1,input_shape=(1,)))
#model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(1))
 
    
# Display a summary of the model architecture.
model.summary()


In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005),
              loss='mae',metrics='RootMeanSquaredError')

In [ ]:
history = model.fit(X_train[:,5:6], 
                    y_train, 
                    batch_size=16, 
                    epochs=100, 
                    validation_split=0.3)

In [ ]:
def plot_loss(history):
    plt.figure(figsize=(20,5))
    plt.plot(history.history['loss'], 'g', label='Training Loss')
    plt.plot(history.history['val_loss'], 'b', label='Validation Loss')
    plt.xlim([0, 100])
    plt.ylim([0, np.max(history.history['loss'])])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

plot_loss(history)   

In [ ]:
np.max(history.history['loss'])

In [ ]:
# Predict the median price of a home with [3, 4, 5, 6, 7] rooms.
x = [3, 4, 5, 6, 7]
y_pred = model.predict(x)
for idx in range(len(x)):
    print("Predicted price of a home with {} rooms: ${}K".format(x[idx], int(y_pred[idx]*10)/10))



In [ ]:
x = tf.linspace(3, 9, 10)
 
# Use the model to predict the dependent variable.
y = model.predict(x)
x

In [ ]:
def plot_data(x_data, y_data, x, y, title=None):
     
    plt.figure(figsize=(15,5))
    plt.scatter(x_data, y_data, label='Ground Truth', color='green', alpha=0.5)
    plt.plot(x, y, color='k', label='Model Predictions')
    plt.xlim([3,9])
    plt.ylim([0,60])
    plt.xlabel('Average Number of Rooms')
    plt.ylabel('Price [$K]')
    plt.title(title)
    plt.grid(True)
    plt.legend()

In [ ]:
plot_data(X_train_1d, y_train, x, y, title='Training Dataset')

In [ ]:
plot_data(X_test_1d, y_test, x, y, title='Test Dataset')